In [2]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import RFE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn .metrics import roc_auc_score
import statistics

f_orgnl - best 12, th=0.435: 0.6993865030674846; rf (): 0.7310657596371882; best k-nn (33): 0.6229326513213981
f_op_1 - best : ; rf: ; best k-nn (): 
f_op_2 - best : ; rf: ; best k-nn (): 
f_op_3 - best : ; rf: ; best k-nn (): 

In [3]:
# Load the data
sti_features = 'train_75_people_data.csv'
#sti_features ='features_optimized_1.csv'
#sti_features ='features_optimized_2.csv'
#sti_features ='features_optimized_3.csv'

data = pd.read_csv(sti_features)

In [4]:
### Field for creating models (and saving them)

def create_knn(x,y,dims,neighbors):
    pca = PCA(n_components=dims)
    pca_component = pca.fit_transform(x)
    final_model = KNeighborsClassifier(n_neighbors=neighbors)
    final_model.fit(pca_component, y)
    pickle.dump(final_model,open(f'knn{neighbors}_{dims}_model.pkl', 'wb'))
    pickle.dump(pca,open(f'pca{dims}.pkl', 'wb'))

def create_rf(x,y,n_trees,depth):
    final_model = RandomForestClassifier(n_estimators=n_trees, max_depth=depth, bootstrap=True)
    final_model.fit(x, y)
    pickle.dump(final_model,open(f'rf{n_trees}_{depth}_model.pkl', 'wb'))

X = data[['H_value', 'S_value', 'V_value', 'red_presence', 'brown_presence', 'blue_presence', 'pink_presence', 'white_presence','black_presence','atypical_pigment_network', 'blue-white_veil', 'asymmetry_values']]
y=data['cancer_or_not']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.T).T

create_knn(X_scaled,y,3,24)
create_rf(X_scaled,y,71,6)

In [17]:
#Field for evaluating the final combined model

def evaluate_model(y,predictions):
    for k in range(len(predictions)):
        predictions[k]=predictions[k]/len(predictions)

    correct = 0
    wrong = 0
    cancer = 0
    not_cancer = 0

    threshold_proba = 0.5

    #Confusion matrix
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    auc_list=[]
    for x in range(len(y)):
        yes = 0
        for k in range(len(predictions)):
            yes+=predictions[k][x][1]
        #yes =  pred_list_model_random_tree_proba_norm[x][1]+pred_list_model_knn_proba_norm[x][1]+pred_list_model_log_proba_norm[x][1]
        if yes > threshold_proba:
            cancer += 1
            if 1 == y[x]:
                TP += 1
                correct += 1
            else:
                FP += 1
                wrong +=1
        else:
            not_cancer += 1
            if 0 == y[x]:
                TN += 1
                correct += 1
            else:
                FN += 1
                wrong +=1
    for k in range(len(predictions)):
        auc_list.append(roc_auc_score(y,predictions[k][:,1]))
    accuracy=correct/(correct+wrong)
    recall=TP/(TP+FN)
    auc=np.mean(auc_list)

    d={"acc":accuracy,"rec":recall,"auc":auc}
    return d

def test(df_test_data,models):
    X_test_data = df_test_data[['H_value', 'S_value', 'V_value', 'red_presence', 'brown_presence', 'blue_presence', 'pink_presence', 'white_presence','black_presence','atypical_pigment_network', 'blue-white_veil', 'asymmetry_values']]
    y_test_data=df_test_data['cancer_or_not'].to_list()

    X_scaled_test_data = scaler_reload.transform(X_test_data)

    X_scaled_pca = pca_reload.transform(X_scaled_test_data)

    predictions=[]
    if 'log' in models:
        predictions.append(model_log.predict_proba(X_scaled_test_data))
    if 'rf' in models:
        predictions.append(model_random_tree.predict_proba(X_scaled_test_data))
    if 'knn' in models:
        predictions.append(model_knn.predict_proba(X_scaled_pca))

    d = evaluate_model(y_test_data,predictions)
    return d

def sampling_test(n,df_test_data_original,models=['log','rf','knn']):
    print(f"Now preforming a sampling test sampling {n} times")
    accuracies=[]
    recalls=[]
    samplesize=len(df_test_data_original)
    for i in range(n):
        df_test_data = df_test_data_original.sample(samplesize,replace=True)
        d=test(df_test_data,models)
        #df_test_data = df_test_data_original
        
        accuracies.append(d["acc"])
        recalls.append(d["rec"])

    print("standard deviation for accuracy",statistics.stdev(accuracies))
    print("mean accuracy ",statistics.mean(accuracies))
    print("standard deviation for recall",statistics.stdev(recalls))
    print("mean recall",statistics.mean(recalls))


model_log = pd.read_pickle('./final_form_of_repository/models/groupR_log_regr_classifier.sav')
model_random_tree = pd.read_pickle('./final_form_of_repository/models/groupR_random_forest_classifier.sav')
model_knn = pd.read_pickle('./final_form_of_repository/models/groupR_knn_classifier.sav')
pca_reload = pd.read_pickle('./final_form_of_repository/models/pca.sav')
df_test_data_original = pd.read_csv('test_25_people_data.csv')
scaler_reload, features = pd.read_pickle('./final_form_of_repository/models/scaler.sav')

#models=['log','rf','knn']
models=['log','rf','knn']
for model in models:
    print(f"Test for {model}: ")
    results=test(df_test_data_original,[model])
    print(f'accuracy: {results["acc"]}\nrecall: {results["rec"]}\nauc:{results["auc"]}')
    sampling_test(1000,df_test_data_original,[model])
    print()


print(f"Test for final model: ")
results=test(df_test_data_original,models)
print(f'accuracy: {results["acc"]}\nrecall: {results["rec"]}\nauc:{results["auc"]}')
sampling_test(1000,df_test_data_original,models)
print()


Test for log: 
accuracy: 0.7008196721311475
recall: 0.7986111111111112
auc:0.7231944444444445
Now preforming a sampling test sampling 1000 times
standard deviation for accuracy 0.02887956263484073
mean accuracy  0.7001475409836065
standard deviation for recall 0.03302622925312891
mean recall 0.7979714821277841

Test for rf: 
accuracy: 0.7336065573770492
recall: 0.8472222222222222
auc:0.7728472222222221
Now preforming a sampling test sampling 1000 times
standard deviation for accuracy 0.029632916966176887
mean accuracy  0.7333770491803279
standard deviation for recall 0.030072257928505514
mean recall 0.8472586716826577

Test for knn: 
accuracy: 0.6516393442622951
recall: 0.7777777777777778
auc:0.6859375
Now preforming a sampling test sampling 1000 times
standard deviation for accuracy 0.030368881275899526
mean accuracy  0.6514262295081967
standard deviation for recall 0.03497393022772075
mean recall 0.7779123703018954

Test for final model: 
accuracy: 0.7049180327868853
recall: 0.819444

In [ ]:
 
if len(sys.argv)==3:
    img=sys.argv[1]
    mask=sys.argv[2]
    print(classify(img,mask))
else:
    print("No input")
    test_csv=pd.read_csv("../test_25_people_data.csv")
    filenames=test_csv["Name_Of_Picture"].to_list()
    y=test_csv["cancer_or_not"].to_list()

    imgs=os.listdir("../images")
    cancer_counter=0
    all_counter=0
    for i in range(len(filenames)):
        
        filename=filenames[i]
        true_value=y[i]

        print(i/len(filenames))

        imgpath="../images/"+filename+".png"
        maskpath="../masks/"+filename+"_mask.png"
        if os.path.exists(maskpath):
            cancer,prob=classify(imgpath,maskpath)
            if cancer==true_value:
                cancer_counter+=1
            all_counter+=1
        else:
            print("nomask")
    print("Accuracy: ",cancer_counter/all_counter)
    

In [ ]:
# Cell for whatever

# 0.10003239426876925
# 0.06920752305228388
'''
n=len(pred_list_model_log_proba)
confusion_matrix = np.matrix([[TN/(TN+FP), FP/(TN+FP)], [FN/(FN+TP), TP/(FN+TP)]])
print('Correct:', correct, '    Wrong:', wrong, '    Cancer prediction:', cancer, '    Not cancer prediction:', not_cancer, '    Accuracy:', )
print("Confusion Matrix:")
print(confusion_matrix)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix,display_labels=model_knn.classes_)
disp.plot()
print(TP/(TP+FN))
#plt.savefig("final_confusion_matrix_prob_classes.png")'''